In [ ]:
use catalog identifier(:catalog);
use schema identifier(:schema);

In [0]:
CREATE OR REPLACE VIEW metv_dbsql
(
  year,
  month,
  date,
  query_count,
  query_duration_sec_avg,
  query_duration_sec_min,
  query_duration_sec_max,
  distinct_users,
  distinct_users_t7d,
  distinct_users_t30d
)
WITH METRICS
LANGUAGE YAML
COMMENT 'A Metric View for DBSQL'
AS $$
version: 0.1

source: users.chris_koester.vw_query_history

joins:
  - name: calendar
    source: users.chris_koester.dim_calendar
    on: calendar.date_int = source.date_int

dimensions:
  - name: year
    expr: calendar.calendar_year
  - name: month
    expr: calendar.calendar_month
  - name: date
    expr: calendar.calendar_date

measures:
  - name: query_count
    expr: count(statement_id)
  - name: query_duration_sec_avg
    expr: avg(total_duration_ms / 1000)
  - name: query_duration_sec_min
    expr: min(total_duration_ms / 1000)
  - name: query_duration_sec_max
    expr: max(total_duration_ms / 1000)
  - name: distinct_users
    expr: count(distinct executed_by_user_id)
  - name: distinct_users_t7d
    expr: distinct_users
    window:
      - order: date
        range: trailing 7 day
        semiadditive: last
  - name: distinct_users_t30d
    expr: distinct_users
    window:
      - order: date
        range: trailing 30 day
        semiadditive: last
$$;